In [ ]:
import cv2
import easyocr
import threading
import queue  # Queue for threading

# Flag to control threads
stop_flag = False

def preprocess_frame(frame):
    """Preprocess the frame for better OCR performance."""
    # Resize the frame to half its size for faster processing
    frame = cv2.resize(frame, (640, 360))  # Resize to 640x360 for better speed
    return frame


def annotate_frame(frame, results):
    """Draw red bounding boxes and labels on the frame."""
    for (bbox, text, prob) in results:
        # Extract bounding box coordinates
        (top_left, top_right, bottom_right, bottom_left) = bbox
        top_left = tuple(map(int, top_left))
        bottom_right = tuple(map(int, bottom_right))

        # Draw a red rectangle around the text
        cv2.rectangle(frame, top_left, bottom_right, (0, 0, 255), 2)

        # Put the detected text above the bounding box
        cv2.putText(
            frame,
            text,
            (top_left[0], max(0, top_left[1] - 10)),  # Avoid negative coordinates
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 0, 255),  # Red color for text
            2,
            cv2.LINE_AA,
        )


def perform_ocr(reader, frame_queue, output_queue):
    """Perform OCR on frames in the queue."""
    while not stop_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()

            # Preprocess the frame for better OCR accuracy
            preprocessed_frame = preprocess_frame(frame)

            # Perform OCR using EasyOCR
            results = reader.readtext(preprocessed_frame)

            # Add OCR results to the output queue
            output_queue.put((preprocessed_frame, results))


def process_frames():
    """Capture frames from the webcam and handle video feed."""
    global stop_flag
    frame_queue = queue.Queue(maxsize=2)  # Limit queue size to prevent overload
    output_queue = queue.Queue(maxsize=2)  # Limit queue size to prevent overload

    # Initialize EasyOCR Reader
    print("Initializing EasyOCR reader...")
    reader = easyocr.Reader(['en'], gpu=True)  # Use GPU if available

    # Start OCR thread
    ocr_thread = threading.Thread(target=perform_ocr, args=(reader, frame_queue, output_queue))
    ocr_thread.start()

    # Open the webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not access the webcam.")
        stop_flag = True
        return

    print("Press 'q' to quit.")
    while not stop_flag:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture video. Exiting...")
            stop_flag = True
            break

        # Add the frame to the processing queue
        if not frame_queue.full():
            frame_queue.put(frame)

        # Check if OCR results are available
        if not output_queue.empty():
            annotated_frame, results = output_queue.get()

            # Annotate the frame with bounding boxes and detected text
            annotate_frame(annotated_frame, results)

            # Display the annotated frame with resized window
            cv2.imshow("Live OCR Feed", annotated_frame)

        # Exit the loop when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_flag = True
            break

    cap.release()
    cv2.destroyAllWindows()
    ocr_thread.join()


def main():
    """Main function to start the program."""
    global stop_flag
    print("Starting live video feed with optimized OCR...")
    process_frames()
    print("Program exited successfully.")


if __name__ == "__main__":
    main()



Starting live video feed with optimized OCR...
Initializing EasyOCR reader...
Press 'q' to quit.
